In [1]:
pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid
from rapidfuzz import fuzz, process


In [3]:
df = pd.read_csv('/kaggle/input/netflix-movies-and-shows-dataset/netflix_data.csv')

In [4]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [5]:
df.tail()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [7]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [8]:
df.duplicated().sum()

0

In [9]:
df['description'].fillna('', inplace=True)
df['listed_in'].fillna('', inplace=True)
df['cast'].fillna('Unknown', inplace=True)
df['director'].fillna('Unknown', inplace=True)

<ipython-input-9-f48a2b1e3158>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['description'].fillna('', inplace=True)
<ipython-input-9-f48a2b1e3158>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me

In [10]:
df['content'] = (
    df['description'] + " " + df['listed_in'] + " " + df['cast'] + " " + df['director']
)

In [11]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,content
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...","As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,Unknown,Unknown,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,In a city of coaching centers known to train I...


In [12]:
df['title'] = df['title'].str.strip().str.lower()

In [13]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1, 2))

tfidf_matrix = tfidf.fit_transform(df['content'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [14]:
def find_fuzzy_match(title, df, limit=3, threshold=70):
    matches = process.extract(title, df['title'], limit=limit, scorer=fuzz.partial_ratio)
    matches = [match for match in matches if match[1] >= threshold]
    return matches

In [15]:
def recommend_content_based(title, df, cosine_sim, top_n=10):
    title = title.strip().lower()

    if title in df['title'].values:
        idx = df[df['title'] == title].index[0]
    else:
        matches = find_fuzzy_match(title, df, limit=1, threshold=70)
        if not matches:
            return f"Judul '{title}' tidak ditemukan dalam dataset."
        title_matched = matches[0][0]
        idx = df[df['title'] == title_matched].index[0]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:top_n+1]
    indices = [i[0] for i in sim_scores]

    recommended_df = df.iloc[indices][['title', 'listed_in', 'description']]
    return recommended_df

In [16]:
param_grid = {
    'max_features': [1000, 3000, 5000],
    'ngram_range': [(1, 1), (1, 2)],
    'stop_words': ['english']
}

best_params = None
best_score = 0
grid = ParameterGrid(param_grid)

In [17]:
for params in grid:
    tfidf = TfidfVectorizer(**params)
    tfidf_matrix = tfidf.fit_transform(df['content'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    title = df['title'].iloc[0]
    recommendations = recommend_content_based(title, df, cosine_sim, top_n=5)

    score = len(recommendations)
    if score > best_score:
        best_score = score
        best_params = params

print("Best Parameters:", best_params)

Best Parameters: {'max_features': 1000, 'ngram_range': (1, 1), 'stop_words': 'english'}


In [18]:
title = "dragon"  
recommendations = recommend_content_based(title, df, cosine_sim, top_n=10)
print("Rekomendasi berdasarkan film:", title)
print(recommendations)

Rekomendasi berdasarkan film: dragon
                             title  \
4595                  needhi singh   
7390                man of tai chi   
295               the paper tigers   
3487               the grandmaster   
7347              love on delivery   
2890  unabomber - in his own words   
3674  pili fantasy: war of dragons   
2916  panipat - the great betrayal   
5356                    love storm   
1344             the last paradiso   

                                              listed_in  \
4595   Action & Adventure, Dramas, International Movies   
7390                         Action & Adventure, Dramas   
295                        Action & Adventure, Comedies   
3487   Action & Adventure, Dramas, International Movies   
7347  Action & Adventure, Comedies, International Mo...   
2890                         Crime TV Shows, Docuseries   
3674  International TV Shows, TV Action & Adventure,...   
2916      Dramas, International Movies, Romantic Movies   
5356  Interna